Most of your work is related to how well this one single function performs:

```python
def predict(data):
  ...
```

To the untrained eye, this looks deceptively simple. Isn't this just one function? I can guarantee you you're about to develop a love-hate relationship with it!

In reality, when dealing with Deep Learning the `predict()` function can have a simple implementation: